In [ ]:
#USEFUL LINKS
# LINK TO REPLACE WORD IN SENTENCE
# https://www.programiz.com/python-programming/methods/string/replace

In [ ]:
#all imports
import pickle
import numpy as np
import pandas as pd

#import tensorflow_hub as hub

from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.7.0'

# DATASET

In [ ]:
data = pd.read_csv('new_data.csv')
print(data.shape)
data.head(5)

(498362, 2)


,correct,incorrect
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang - 8 was ..."
2,His Kanji ability is much better than mine .,His Kanji 's ability is much better than me .
3,"We 've known each other for only half a year ,...","We 've known each other for only half a year ,..."
4,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .


In [ ]:
# ckeck if any missing value is present
data.isnull().values.any()

True

In [ ]:
data.dropna(inplace=True)
data.shape

(498360, 2)

In [ ]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True,drop=True)
data.shape

(496339, 2)

In [ ]:
def clean(text):
    """
    takes string as input and
    removes characters inside (),{},[] and <>
    removes characters like -+@#^/|*(){}$~`
    we not not removing ,.!-:;"' as these characters are present in english language 
    """
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text

data["correct"] = data["correct"].apply(clean)
data["incorrect"] = data["incorrect"].apply(clean)

In [ ]:
def percentile(low,high,step,list_temp):
    """
    this function takes low, high, step size as input and prints percentiles accordingly
    """
    for i in np.arange(low,high,step):
        print(i,"percentile is ",np.percentile(list_temp ,i))

In [ ]:
def sen_to_char(sen):
    return len([i for i in sen])

corr_length = data["correct"].apply(sen_to_char)
corr_length = list(corr_length)
len(corr_length)

496339

In [ ]:
percentile(0,101,10,corr_length)
print("***************************************************************")
percentile(90,101,1,corr_length)

0 percentile is  0.0
10 percentile is  27.0
20 percentile is  35.0
30 percentile is  42.0
40 percentile is  48.0
50 percentile is  55.0
60 percentile is  63.0
70 percentile is  73.0
80 percentile is  86.0
90 percentile is  108.0
100 percentile is  2622.0
***************************************************************
90 percentile is  108.0
91 percentile is  112.0
92 percentile is  116.0
93 percentile is  120.0
94 percentile is  125.0
95 percentile is  131.0
96 percentile is  139.0
97 percentile is  149.0
98 percentile is  163.0
99 percentile is  188.0
100 percentile is  2622.0


In [ ]:
# removing those data points which have correct sentence of length more than 108
index = []
for i in range(len(corr_length)):
    if corr_length[i] > 108:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(446785, 2)


In [ ]:
incorr_length = data["incorrect"].apply(sen_to_char)
incorr_length = list(incorr_length)
len(incorr_length)

446785

In [ ]:
percentile(0,101,10,incorr_length)
print("***********************************************************")
percentile(90,101,1,incorr_length)

0 percentile is  0.0
10 percentile is  25.0
20 percentile is  32.0
30 percentile is  38.0
40 percentile is  44.0
50 percentile is  50.0
60 percentile is  57.0
70 percentile is  64.0
80 percentile is  73.0
90 percentile is  85.0
100 percentile is  611.0
***********************************************************
90 percentile is  85.0
91 percentile is  87.0
92 percentile is  89.0
93 percentile is  91.0
94 percentile is  92.0
95 percentile is  95.0
96 percentile is  97.0
97 percentile is  99.0
98 percentile is  103.0
99 percentile is  107.0
100 percentile is  611.0


In [ ]:
# removing those data points which have incorrect sentence of length more than 108
index = []
for i in range(len(incorr_length)):
    if incorr_length[i] > 108:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(443397, 2)


In [ ]:
corr_length = data["correct"].str.split().apply(len)
corr_length = list(corr_length)

In [ ]:
percentile(0,101,10,corr_length)
print("***************************************************************")
percentile(90,101,1,corr_length)

0 percentile is  0.0
10 percentile is  6.0
20 percentile is  8.0
30 percentile is  9.0
40 percentile is  10.0
50 percentile is  11.0
60 percentile is  13.0
70 percentile is  14.0
80 percentile is  16.0
90 percentile is  18.0
100 percentile is  47.0
***************************************************************
90 percentile is  18.0
91 percentile is  19.0
92 percentile is  19.0
93 percentile is  19.0
94 percentile is  20.0
95 percentile is  20.0
96 percentile is  21.0
97 percentile is  21.0
98 percentile is  22.0
99 percentile is  23.0
100 percentile is  47.0


In [ ]:
# removing those data points which have correct sentence of length more than 24
index = []
for i in range(len(corr_length)):
    if corr_length[i] > 24:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(441931, 2)


In [ ]:
incorr_length = data["incorrect"].str.split().apply(len)
incorr_length = list(incorr_length)

In [ ]:
percentile(0,101,10,incorr_length)
print("***************************************************************")
percentile(90,101,1,incorr_length)

0 percentile is  0.0
10 percentile is  6.0
20 percentile is  7.0
30 percentile is  8.0
40 percentile is  10.0
50 percentile is  11.0
60 percentile is  12.0
70 percentile is  14.0
80 percentile is  15.0
90 percentile is  18.0
100 percentile is  31.0
***************************************************************
90 percentile is  18.0
91 percentile is  18.0
92 percentile is  18.0
93 percentile is  19.0
94 percentile is  19.0
95 percentile is  20.0
96 percentile is  20.0
97 percentile is  21.0
98 percentile is  21.0
99 percentile is  22.0
100 percentile is  31.0


In [ ]:
# removing those data points which have incorrect sentence of length more than 25
index = []
for i in range(len(incorr_length)):
    if incorr_length[i] > 25:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(441842, 2)


In [ ]:
data["correct"] = data["correct"].astype(str)
data["incorrect"] = data["incorrect"].astype(str)

In [ ]:
type(data.correct) , data.isnull().sum().sum()

(pandas.core.series.Series, 0)

In [ ]:
length_max = max([len(i) for i in tqdm(data['correct'])])
length_min = min([len(i) for i in tqdm(data['correct'])])
avg = [len(i) for i in tqdm(data['correct'])]
length_avg = np.array([avg]).mean()

length_max , length_min , length_avg

100%|█████████████████████████████████████████████████████████████████████| 441842/441842 [00:00<00:00, 3355215.39it/s]


(108, 0, 53.660100216819586)

In [ ]:
CORRECT_SENTENCE_LEN = data['correct'].str.split().apply(len) 
ERRONEOUS_SENTENCE_LEN = data['incorrect'].str.split().apply(len)

In [ ]:
for i in range(0,101,10):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in range(90,101):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in [99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100]:
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))

0 0.0
10 6.0
20 7.0
30 8.0
40 10.0
50 11.0
60 12.0
70 14.0
80 15.0
90 18.0
100 25.0
90 18.0
91 18.0
92 18.0
93 19.0
94 19.0
95 20.0
96 20.0
97 21.0
98 21.0
99 22.0
100 25.0
99.1 22.0
99.2 23.0
99.3 23.0
99.4 23.0
99.5 23.0
99.6 23.0
99.7 24.0
99.8 24.0
99.9 24.0
100 25.0


SINCE 99.9% OF DATA HAS LENGTH LESS THAN 10 , SO SELECTING SENTENCE WITH WORD <10

In [ ]:
data['CORRECT_SENTENCE_LEN'] = data['correct'].str.split().apply(len)
data = data[data['CORRECT_SENTENCE_LEN'] < 20]

data['ERRONEOUS_SENTENCE_LEN'] = data['incorrect'].str.split().apply(len)
data = data[data['ERRONEOUS_SENTENCE_LEN'] < 20]

#ADDING start and end IN THE SENTENCES
data['english_inp'] = '<start> ' + data['correct'].astype(str)
data['english_out'] = data['correct'].astype(str) + ' <end>'

data = data.drop(['correct','CORRECT_SENTENCE_LEN','ERRONEOUS_SENTENCE_LEN'], axis=1)
print(data.shape)
data.head()

(409715, 3)


,incorrect,english_inp,english_out
0,And he took in my favorite subject like soccer .,<start> And he took in my favorite subjects li...,And he took in my favorite subjects like socce...
1,"Actually , who let me know about Lang was him .","<start> Actually , he was the one who let me k...","Actually , he was the one who let me know abou..."
2,His Kanji 's ability is much better than me .,<start> His Kanji ability is much better than ...,His Kanji ability is much better than mine . <...
4,I heard a sentence last night when I watched TV .,<start> I heard a sentence last night when I w...,I heard a sentence last night when I was watch...
5,"When you go downhill , you have to stick out y...","<start> When you go downhill , you have to sti...","When you go downhill , you have to stick out y..."


### Getting train and test 

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2)

In [ ]:
print(train.shape, validation.shape)
#ADDING TO <end> TO ONE OF THE SENTENCES SO THAT TOKENIZER LEARNS THE WORD <end>
train.iloc[0]['english_inp']= str(train.iloc[0]['english_inp'])+' <end>'
train.iloc[0]['english_out']= str(train.iloc[0]['english_out'])+' <end>'

(327772, 3) (81943, 3)


In [ ]:
max_encoder_length = max([len(i) for i in tqdm(train['incorrect'])])
max_decoder_length = max([len(i) for i in tqdm(train['english_inp'])])
print(max_encoder_length , max_decoder_length)

100%|█████████████████████████████████████████████████████████████████████| 327772/327772 [00:00<00:00, 2533625.15it/s]

108 116


In [ ]:
train.sample(2)

,incorrect,english_inp,english_out
266458,"Last week , my university did graduation .","<start> Last week , my university had their gr...","Last week , my university had their graduation..."
404826,"When I was little , she always made clothes fo...","<start> When I was little , she always made cl...","When I was little , she always made clothes fo..."


In [ ]:
validation.sample(2)

,incorrect,english_inp,english_out
177050,Now even I 'm thinking about studying abroad i...,<start> Now I 'm even thinking about studying ...,Now I 'm even thinking about studying abroad i...
238422,So I do n't know this system .,<start> So I do n't know how this system works .,So I do n't know how this system works . <end>


**TOKENIING**

In [ ]:
tknizer_ERRONEOUS_SENTENCE = Tokenizer()
tknizer_ERRONEOUS_SENTENCE.fit_on_texts(train['incorrect'].values)
tknizer_CORRECT_SENTENCE = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_CORRECT_SENTENCE.fit_on_texts(train['english_inp'].values)

In [ ]:
vocab_size_CORRECT_SENTENCE=len(tknizer_CORRECT_SENTENCE.word_index.keys())
print(vocab_size_CORRECT_SENTENCE)
vocab_size_ERRONEOUS_SENTENCE=len(tknizer_ERRONEOUS_SENTENCE.word_index.keys())
print(vocab_size_ERRONEOUS_SENTENCE)

45162
58998


In [ ]:
tknizer_CORRECT_SENTENCE.word_index['<start>'], tknizer_CORRECT_SENTENCE.word_index['<end>']

(1, 23129)

TOKENIZER WITH ENGLISH WORDS

In [ ]:
embeddings_index = dict()
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size_CORRECT_SENTENCE+1, 100))
for word, i in tknizer_CORRECT_SENTENCE.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(45163, 100)

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c


In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
    
    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
        
    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

## Data pipeline

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['incorrect'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)




In [ ]:
class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

## Model

In [ ]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() 
        self.encoder = Encoder(vocab_size=vocab_size_ERRONEOUS_SENTENCE + 1, embedding_dim=100, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=vocab_size_CORRECT_SENTENCE + 1, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [ ]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
vanilla = vanilla_model(encoder_inputs_length=16,decoder_inputs_length=16,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)
vanilla.compile(optimizer= optimizer, loss= loss_function, metrics=['accuracy'])

In [ ]:
import datetime

# logfile = "/content/Project_2/logs/vanilla/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# #logfile = "C:\\Users\\rpris\\Google Drive\\Project_2\\logs\\vanilla\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tfboard = tf.keras.callbacks.TensorBoard(log_dir=logfile, histogram_freq=1, write_graph=True)

# chkfile = "/content/wts/vanilla/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
# #chkfile2 = "C:\\Users\\rpris\\Google Drive\\Project_2\\wts\\vanilla\\weights-{epoch:02d}-{val_loss:.4f}.hdf5"
# chkpt = tf.keras.callbacks.ModelCheckpoint(chkfile, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')

# stp = tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss')

In [ ]:
# vanilla.load_weights("/content/drive/MyDrive/Project_2/wts/vanilla/weights-65-0.5898.hdf5")

In [ ]:
# vanilla.summary()

In [ ]:
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
print(train_steps,valid_steps)

320 80


In [ ]:
vanilla  = vanilla_model(encoder_inputs_length=20,decoder_inputs_length=20,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
vanilla.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 50 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=10 , validation_data=train_dataloader, validation_steps=valid_steps )#, callbacks=[stp, chkpt, tfboard]
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
vanilla.summary()

C:\Users\kiran\AppData\Local\Temp\ipykernel_25572\871002888.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=10 , validation_data=train_dataloader, validation_steps=valid_steps )#, callbacks=[stp, chkpt, tfboard]


Epoch 1/10
320/320 [==============================] - 2499s 8s/step - loss: 3.2887 - val_loss: 3.0354
Epoch 2/10
320/320 [==============================] - 3216s 10s/step - loss: 2.8410 - val_loss: 2.6673
Epoch 3/10
320/320 [==============================] - 2655s 8s/step - loss: 2.5385 - val_loss: 2.4194
Epoch 4/10
320/320 [==============================] - 2798s 9s/step - loss: 2.3336 - val_loss: 2.2396
Epoch 5/10
320/320 [==============================] - 2801s 9s/step - loss: 2.1617 - val_loss: 2.0676
Epoch 6/10
320/320 [==============================] - 2749s 9s/step - loss: 1.9889 - val_loss: 1.9018
Epoch 7/10
320/320 [==============================] - 2823s 9s/step - loss: 1.8417 - val_loss: 1.7641
Epoch 8/10
320/320 [==============================] - 2665s 8s/step - loss: 1.7206 - val_loss: 1.6478
Epoch 9/10
320/320 [==============================] - 2676s 8s/step - loss: 1.6114 - val_loss: 1.5484
Epoch 10/10
320/320 [==============================] - 2672s 8s/step - loss: 1.51

In [ ]:
os.mkdir('saved_model_newww')

In [ ]:
vanilla.save_weights('saved_model_newww/vanilla.h5')

In [ ]:
vanilla  = vanilla_model(encoder_inputs_length=20,decoder_inputs_length=20,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
vanilla.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 50 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("best_Model.h5",save_best_only=True)
vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=1 , validation_data=train_dataloader, validation_steps=valid_steps\
                      , callbacks=[model_checkpoint_callback])#, callbacks=[stp, chkpt, tfboard]
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
vanilla.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


113/113 [==============================] - 92s 738ms/step - loss: 4.9768 - val_loss: 4.1294
Model: "vanilla_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_4 (Encoder)          multiple                  10858468  
_________________________________________________________________
decoder_4 (Decoder)          multiple                  3798768   
_________________________________________________________________
dense_4 (Dense)              multiple                  8823067   
Total params: 23,480,303
Trainable params: 23,480,303
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vanilla.load_weights("//content/drive/MyDrive/saved_model_newww/vanilla.h5")

In [ ]:
vanilla.summary()

Model: "vanilla_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  6265468   
                                                                 
 decoder_2 (Decoder)         multiple                  4881868   
                                                                 
 dense_2 (Dense)             multiple                  11606634  
                                                                 
Total params: 22,753,970
Trainable params: 22,753,970
Non-trainable params: 0
_________________________________________________________________


## Inference

In [ ]:
train.incorrect[1] , train.english_inp[1] , 

('Actually , who let me know about Lang   was him .',
 '<start> Actually , he was the one who let me know about Lang   . .')

In [ ]:
enc_inp = train.incorrect[1]
dec_inp =  train.english_inp[1] 

In [ ]:
translation=""

e_input=[]
for i in enc_inp.split():
    if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
        e_input.append(0)
    else:
        e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

#e_input = pad_sequences(e_input, maxlen=16, padding='post')


e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

#there is no onestep decoder in this thing, so I have to use the decoder input to predict output

#decoder input
d_input=[]
for i in dec_inp.split():
    if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
        d_input.append(0)
    else:
        d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

#d_input = pad_sequences(d_input, maxlen=16, padding='post')

prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

for word in prediction[0]:
    word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
    if word == "<end>":
        break
    translation += word + " "
translation

'please please please wants trying first wants wants you see about '

In [ ]:
sample_train = train.sample(1000)
sample_train

,incorrect,english_inp,english_out
436016,Launch was a barbecue .,<start> Lunch was a barbecue .,Lunch was a barbecue . <end>
158251,"I have technical blog , blog at tumblr where I...",<start> I have a technical blog : a blog at tu...,I have a technical blog : a blog at tumblr whe...
54934,But the people lined the back of me lend me mo...,<start> But the person in line behind me offer...,But the person in line behind me offered to le...
13291,"Also in Japan , there had been a cat which sai...","<start> Also in Japan , there had been a cat w...","Also in Japan , there had been a cat which sai..."
40676,"According to Lind , why did NOT Chunky Monkey ...","<start> According to Lind , why did Chunky Mon...","According to Lind , why did Chunky Monkey NOT ..."
...,...,...,...
279767,"After that , I 'll study English and watching ...","<start> After that , I 'll study English and w...","After that , I 'll study English and watch mov..."
441661,I do n't know what to do to go through the mis...,<start> I do n't know what to do to survive th...,I do n't know what to do to survive this miser...
236406,Anyway we really wish we could help them .,<start> Anyway we really wished that we could ...,Anyway we really wished that we could help the...
388531,The look of it is very wild .,<start> It looked very wild .,It looked very wild . <end>


In [ ]:
def inference(enc_inp,dec_inp):
            
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
        translation += word + " "
    return translation
a = train.incorrect[5]
b =  train.english_inp[5]
pred = inference(a,b)
pred

'you you should to you you have to your your your face or you '

In [ ]:
a = train.incorrect[10]
b =  train.english_inp[10]
pred = inference(a,b)
a  , b , pred

('The making souvenir is a hard and interesting work .',
 '<start> Making souvenirs is a hard but interesting work .',
 'a a is a good job a and ')

In [ ]:
a = train.incorrect[12]
b =  train.english_inp[12]
pred = inference(a,b)
a  , b , pred

("That 's two balls : one with a cat inside , other with dog .",
 '<start> There are two balls : one with a cat inside , the other with dog .',
 'two two two two a a piece a lot with of of and and her ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[200]
b =  train.english_inp[200]
pred = inference(a,b)
a  , b , pred

('He spoke with such self confixence that his hearers coulx',
 '<start> He spoke with such self confidence that his hearers could',
 'spoke spoke with such self confidence that the hearers thought ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[300]
b =  train.english_inp[300]
pred = inference(a,b)
a  , b , pred

('The sovereigns Why y are sending ambassadors to compliment usurper',
 '<start> The sovereigns Why they are sending ambassadors to compliment the',
 'negroes negroes they they are sending to to compliment ease honourable ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[500]
b =  train.english_inp[500]
pred = inference(a,b)
a  , b , pred

('What is it you are afraid of Lise I don',
 '<start> What is it you are afraid of Lise I don',
 'is is it you are afraid of don don don ')

In [ ]:
a = train.ERRONEOUS_SENTENCE[1000]
b =  train.english_inp[1000]
pred = inference(a,b)
a  , b , pred

('Secrets indeed All have secrets ff their fwn answered Natasha',
 '<start> Secrets indeed All have secrets of their own answered Natasha',
 'indeed indeed you you your of their own answered ')

In [ ]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = inference(i , j)
    result.append(pred)

result

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.10it/s]


['do do do do think a favorite ',
 "tell tell tell me that it 's on the way but i way is not ",
 'a a a in school school ',
 'but the weather off cold cold ',
 'they they to talk to english english english ',
 'a a of is is is been to the the ',
 'why was the the the earthquake of the earthquake ',
 'i i to do my husband ',
 'it it the problem about the world ',
 'the the is is is ',
 'you you can you you you to to like also to this very ',
 'was was i i i me ',
 'i i years year old ',
 'i i i tired ',
 "did did n't know know how to know the job and it to 's ",
 'i i i me so ',
 'it it really sad ',
 'the the is really nice ',
 'i i trying to write a diary for in write ',
 'there there there is a to me me me ',
 'today today was the last lesson with my english english was was ',
 "do do n't have to have ",
 'it it a to be ',
 'going going from university university i i i to study abroad ',
 'is is the event of the world of in ',
 'nobody nobody that that because because of people are t

In [ ]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,incorrect,english_inp,english_out,PREDICTED_SENTENCE
14235,"So , I must think fashion sense .","<start> So , I must develop my fashion sense .","So , I must develop my fashion sense . <end>",do do do do think a favorite
108585,This saying tell me that it depends on your he...,<start> This saying tells me that it depends o...,This saying tells me that it depends on your h...,tell tell tell me that it 's on the way but i ...
322045,She graduates an elementary school .,<start> She is graduating from elementary scho...,She is graduating from elementary school . <end>,a a a in school school
289399,but the day getting cold . . .,<start> but the days are getting colder . . .,but the days are getting colder . . . <end>,but the weather off cold cold
193771,And they started to talk in an African language .,<start> They started to talk in an African lan...,They started to talk in an African language . ...,they they to talk to english english english
156072,"For some reason , I came to like active sports .","<start> For some reason , I have come to like ...","For some reason , I have come to like active s...",a a of is is is been to the the
66049,why were I in the middle of the street ?,<start> why was I in the middle of the street ?,why was I in the middle of the street ? <end>,why was the the the earthquake of the earthquake
124938,I want to kill my mom !,<start> I want to kill my mom !,I want to kill my mom ! <end>,i i to do my husband
304714,it would be biggest question all the world .,<start> would be biggest question in the world .,would be biggest question in the world . <end>,it it the problem about the world
107039,The laptop of DELL is breakable .,<start> The laptop of DELL is not durable .,The laptop of DELL is not durable . <end>,the the is is is


## BLEU Score

In [ ]:
import nltk

In [ ]:
inp = sample_train.incorrect.values[20]
translate = sample_train.PREDICTED_SENTENCE.values[20]
print(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

And today , it was last lesson by my english teacher .
today today was the last lesson with my english english was was 
BLEU score: 1.2676114249651737e-231


C:\Users\kiran\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\kiran\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\kiran\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

In [ ]:
import nltk.translate.bleu_score as bleu

score = 0
for i in tqdm(range(1000)):
    inp = sample_train.incorrect.values[i]

    # print(inp)
    translate = sample_train.PREDICTED_SENTENCE.values[i]
    # print(translate)
    bleu = nltk.translate.bleu_score.sentence_bleu(inp, translate)
    score = score + bleu
average_bleu = score / 1000
print('the average BLEU score of these sentences. are :' , average_bleu)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 521.14it/s]

the average BLEU score of these sentences. are : 1.3058902476517232e-231


# ATTENTION

In [ ]:
#all imports
import pickle
import numpy as np
import pandas as pd

#import tensorflow_hub as hub
from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.7.0'

### <font color='blue'>**Implement custom encoder decoder and attention layers**</font>

<font color='blue'>**Encoder**</font>

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()
        self.inp_vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
        self.embedding = Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):
        input_embedd = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd, states)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    
    def initialize_states(self,batch_size):
      self.lstm_state_h = tf.zeros([batch_size , self.lstm_size])
      self.lstm_state_c = tf.zeros([batch_size , self.lstm_size])
      return self.lstm_state_h,self.lstm_state_c


<font color='blue'>**Attention**</font>

In [ ]:
class Attention(tf.keras.layers.Layer):

  def __init__(self,scoring_function, att_units):
    super(Attention, self).__init__()
    self.scoring_function = scoring_function
    self.att_units = att_units
    self.W1 = tf.keras.layers.Dense(att_units)
    self.W2 = tf.keras.layers.Dense(att_units)
    self.V = tf.keras.layers.Dense(1)

  
  def call(self,decoder_hidden_state,encoder_output):
    
    if self.scoring_function == 'dot':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))

        #I WAS USING tf.keras.layers.Dot FOR DOT PRODUCT , BUT IT GAVE INCOMPATIBILITY IN SHAPES , SO NOW I VE USED tf.keras.layers.dot
        score =  tf.keras.layers.dot([ encoder_output , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'general':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))
        W = tf.random.uniform(shape=[encoder_output.shape[0] , self.att_units , self.att_units])
        score =  tf.keras.layers.dot([ encoder_output , W] , [2,1]) 
        score =  tf.keras.layers.dot([ score , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'concat':

        decoder_hidden_state_reshaped = tf.expand_dims(decoder_hidden_state, 1)
        score =  self.V(tf.nn.tanh(self.W1(decoder_hidden_state_reshaped) + self.W2(encoder_output)) )
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    
    

<font color='blue'>**OneStepDecoder**</font>

In [ ]:
class One_Step_Decoder(tf.keras.Model):
    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(One_Step_Decoder, self).__init__()

        # Initialize decoder embedding layer, LSTM 
        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.attention=Attention(score_fun,att_units)
        self.embedding = tf.keras.layers.Embedding(tar_vocab_size, embedding_dim)
        self.lstm = LSTM(self.dec_units , return_state=True, return_sequences=True, name="Decoder_LSTM")
        self.dense = tf.keras.layers.Dense(self.tar_vocab_size)

    def call(self,input_to_decoder, encoder_output, state_h,state_c):
        output = self.embedding(input_to_decoder) # (32, 1, 12)
        context_vector,attention_weights=self.attention(state_h,encoder_output)
        concat = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)
        lstm_output, state_h, state_c = self.lstm(concat)
        
        output = self.dense(lstm_output)
        output = tf.reshape(output, (-1, output.shape[2]))

        return output,state_h,state_c,attention_weights,context_vector


<font color='blue'>**Decoder**</font>

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(Decoder , self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.onestep_decoder=One_Step_Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)

    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        all_outputs = tf.TensorArray(tf.float32, size=tf.shape(input_to_decoder)[1])
        for timestep in range(tf.shape(input_to_decoder)[1]):
            output,state_h,state_c,attention_weights,context_vector = self.onestep_decoder(input_to_decoder[: , timestep : timestep + 1] , \
                                                                                           encoder_output , decoder_hidden_state , decoder_cell_state)


            all_outputs = all_outputs.write(timestep , output)
        all_outputs = tf.transpose(all_outputs.stack() , [1,0,2])
        return all_outputs




In [ ]:
out_vocab_size=13 
embedding_dim=12 
input_length=10
dec_units=16 
att_units=16
batch_size=32

target_sentences=tf.random.uniform(shape=(batch_size,input_length),maxval=10,minval=0,dtype=tf.int32)
encoder_output=tf.random.uniform(shape=[batch_size,input_length,dec_units])
state_h=tf.random.uniform(shape=[batch_size,dec_units])
state_c=tf.random.uniform(shape=[batch_size,dec_units])
score_fun = 'concat'
decoder=Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
output=decoder(target_sentences,encoder_output, state_h, state_c)
print(output.shape)

(32, 10, 13)


<font color='blue'>**Encoder Decoder model**</font>

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['incorrect'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)


class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [ ]:
class encoder_decoder(tf.keras.Model):
    def __init__(self,score_fun , encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__()
        #encoder decoder

        self.score_fun = score_fun

        self.encoder=Encoder(inp_vocab_size = vocab_size_ERRONEOUS_SENTENCE+1,embedding_size = 50,lstm_size = 64,input_length = encoder_inputs_length)
        self.decoder=Decoder(out_vocab_size = vocab_size_CORRECT_SENTENCE+1, embedding_dim = 100, input_length = decoder_inputs_length, dec_units =  64 \
                             ,score_fun = self.score_fun ,att_units = 64)

    def call(self,data):

        input,output = data[0], data[1]
        initial_state= self.encoder.initialize_states(batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input , initial_state)

        decoder_output= self.decoder(output,encoder_output, encoder_h, encoder_c)

        return decoder_output


In [ ]:
batch_size=512
score_fun  = 'general'
att_units = 64
model_2  = encoder_decoder(score_fun = score_fun , encoder_inputs_length=20,decoder_inputs_length=10,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
model_2.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 20 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
log_dir="logs1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model_2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=15, validation_data=train_dataloader, validation_steps=valid_steps , callbacks = checkpoint)
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
model_2.summary()

C:\Users\kiran\AppData\Local\Temp\ipykernel_25572\2738313064.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=15, validation_data=train_dataloader, validation_steps=valid_steps , callbacks = checkpoint)


Epoch 1/15
320/320 [==============================] - 4301s 13s/step - loss: 4.2143 - val_loss: 3.8058
Epoch 2/15
 64/320 [=====>........................] - ETA: 54:48 - loss: 3.9147

KeyboardInterrupt: 

In [ ]:
os.mkdir('saved_model_attention')

In [ ]:
model_2.save_weights('saved_model_attention/attention.h5')

In [ ]:
model_2.load_weights("/content/drive/MyDrive/saved_model_attention/attention.h5")
model_2.summary()

Model: "encoder_decoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  4347640   
_________________________________________________________________
decoder_2 (Decoder)          multiple                  5753269   
Total params: 10,100,909
Trainable params: 10,100,909
Non-trainable params: 0
_________________________________________________________________


## <font color='blue'>**Inference**</font>

<font color='blue'>**CORRECTING THE INCORRECT SENTENCES**</font>

In [ ]:
def predict(input_sentence):
    pred = []
    input_sequence = tknizer_CORRECT_SENTENCE.texts_to_sequences([input_sentence])
    result = ' '
    encoder_seq = pad_sequences(input_sequence, maxlen=20, dtype='int32', padding='post')  
    initial_state = model_2.layers[0].initialize_states(1)
    encoder_output, encoder_h, encoder_c = model_2.layers[0](tf.constant(encoder_seq), initial_state)
    start_index = tf.constant([[tknizer_CORRECT_SENTENCE.word_index['<start>']]])
    states = [encoder_h, encoder_c]

    for i in range(20): 
        # print(start_index)
        predicted_out,state_h,state_c,attention_weights,context_vector = model_2.layers[1].onestep_decoder(start_index, encoder_output , encoder_h, encoder_c )
        attention_weights = tf.reshape(attention_weights, (-1, ))
        start_index = np.reshape(np.argmax(predicted_out), (1, 1))
        pred.append(tknizer_CORRECT_SENTENCE.index_word[start_index[0][0]])
        if(pred[-1]=='<end>'):
            break
        translated_sentence = ' '.join(pred)
    return translated_sentence

inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

These wire works was made by my friend of next seat in university .
i will meet their first i will meet their first i will meet their first i will meet their first
BLEU score: 1.0377133938315695e-231


In [ ]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

At  .  a . m .
i want
BLEU score: 1.5319719891192393e-231


In [ ]:
inp = validation.values[20][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

How can I do for she ? ? ? ?
i will have as i will have as i will have as i will have as i will have as
BLEU score: 9.72161026064145e-232


In [ ]:
inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

These wire works was made by my friend of next seat in university .
i will meet their first i will meet their first i will meet their first i will meet their first
BLEU score: 1.0377133938315695e-231


In [ ]:
#USING THIS METHOD TO GET THE BLEU SCORE , THE REFERENCE NOTEBOOK METHOD SHOWS ME AN ERROR FOR 1000 FILES , BUT IT WORKS FINE FOR SINGLE FILE
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

He s just started at the Ministry and this is
i have the
BLEU score: 0.8801117367933934


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = predict(i)
    result.append(pred)

result

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:22<00:00, 12.12it/s]


['i want to make a as a as a as a as a as a as a as a as',
 'i want',
 'i want',
 'i want',
 'i will have their first i will have their first i will have their first i will have their first',
 'i want',
 'i want',
 'i will have their first i will have their first i will have their first i will have their first',
 'i want',
 'i want',
 'i want',
 'i will meet',
 'i will meet their first i will meet their first i will meet their first i will meet their first',
 'i will meet',
 'i want',
 'i will meet',
 'i want to make a as i want to make a as i want to make a as i want',
 'i will meet',
 'i will meet their as i will meet their as i will meet their as i will meet their as',
 'i want',
 'i want',
 'i will meet their first i will meet their first i will meet their first i will meet their first',
 'i want',
 'i want',
 'i will meet their first i will meet their first i will meet their first i will meet their first',
 'i want',
 'i want',
 'i will meet',
 'i want',
 'i want',
 'i want',
 'i 

In [ ]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,incorrect,english_inp,english_out,PREDICTED_SENTENCE
177186,One of my girlfriends terribly frightened,<start> One of my girlfriends was terribly fri...,One of my girlfriends was terribly frightened ...,i want to make a as a as a as a as a as a as a...
181690,She becomes two grades from this April .,<start> She join the second grade from this Ap...,She join the second grade from this April . <end>,i want
275101,Stickam is a web service where we can broadcas...,<start> Stickam is a web service where we can ...,Stickam is a web service where we can broadcas...,i want
251982,I feeled very sad .,<start> I felt very sad .,I felt very sad . <end>,i want
27655,"The night of the day , I do n't slept because ...","<start> At night , I was n't able to sleep bec...","At night , I was n't able to sleep because I t...",i will have their first i will have their firs...
383240,"From this generation , they could speak Englis...","<start> From this generation , they could spea...","From this generation , they could speak Englis...",i want
350801,and I 'll write congratulations letter to her .,<start> and I 'll write a congratulations lett...,and I 'll write a congratulations letter for h...,i want
221737,some were too hot .,<start> Some was too hot .,Some was too hot . <end>,i will have their first i will have their firs...
149709,OMG ! ! ! I almost missed the th . Chinese Bri...,<start> OMG ! ! ! I almost missed the th Chin...,OMG ! ! ! I almost missed the th Chinese Brid...,i want
108845,"Today , e mail become estalished among many ...","<start> Today , e mail becomes estalished am...","Today , e mail becomes estalished among many...",i want


<font color='blue'>**Calculate BLEU score**</font>

In [ ]:
#Create an object of your custom model.
#Compile and train your model on dot scoring function.
# Visualize few sentences randomly in Test data
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

#Sample example
import nltk.translate.bleu_score as bleu
reference = ['i am groot'.split(),] # the original
translation = 'it is ship'.split() # trasilated using model
print('BLEU score: {}'.format(bleu.sentence_bleu(reference, translation)))

BLEU score: 0
